In [1]:
# On importe les pacakges necessaire

from torchvision import datasets, models, transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
from PIL import Image
import torch
import time
import cv2
import os


In [2]:
# telechargement des données
# Où les données seront stockées

data_path = '/home/cross/Fab2Alex.github.io/Simple_CNN_en_pytorch/'

# Training set
cifar10 = datasets.CIFAR10(data_path, train=True, download=True)
# Val set
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# chargement les données
# et transformations des données (en tenseur et normalisées)
cifar10 = datasets.CIFAR10(data_path, train=True, download=False,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.4915, 0.4823, 0.4468),
                                                   (0.2470, 0.2435, 0.2616))
                          ]))

cifar10_val = datasets.CIFAR10(data_path, train=False, download=False,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.4915, 0.4823, 0.4468),
                                                   (0.2470, 0.2435, 0.2616))
                          ]))

In [4]:
# on prends les données et on les stockent ds une variable en precisant le Batch size
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle = False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle = False)

In [5]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [81]:
# À quoi ressemblent nos données?
for i,data in enumerate(train_loader):
    print(data)
    break

[tensor([[[[-1.0531e+00, -1.3072e+00, -1.1960e+00,  ...,  5.1866e-01,
            4.2339e-01,  3.5989e-01],
          [-1.7358e+00, -1.9899e+00, -1.7041e+00,  ..., -3.7033e-02,
           -1.0054e-01, -5.2909e-02],
          [-1.5930e+00, -1.7358e+00, -1.2119e+00,  ..., -1.1642e-01,
           -8.4663e-02, -2.5931e-01],
          ...,
          [ 1.3125e+00,  1.2014e+00,  1.1537e+00,  ...,  5.5041e-01,
           -1.1008e+00, -1.1484e+00],
          [ 8.6794e-01,  7.5681e-01,  9.6321e-01,  ...,  9.3145e-01,
           -4.4983e-01, -6.7210e-01],
          [ 8.2031e-01,  6.7742e-01,  8.5207e-01,  ...,  1.4395e+00,
            4.0752e-01, -3.7033e-02]],

         [[-9.8219e-01, -1.2399e+00, -1.2077e+00,  ...,  1.4516e-01,
            3.2427e-02,  1.6322e-02],
          [-1.6586e+00, -1.9807e+00, -1.8519e+00,  ..., -5.6346e-01,
           -6.4398e-01, -5.7956e-01],
          [-1.5942e+00, -1.8680e+00, -1.5459e+00,  ..., -6.2788e-01,
           -6.2788e-01, -8.0503e-01],
          ...,
    

In [82]:
# petit rappel de resizing avec pytorch.
y = torch.rand([2,5])
print(y)

tensor([[0.8352, 0.4473, 0.0355, 0.9228, 0.3016],
        [0.5993, 0.8486, 0.0842, 0.9758, 0.8946]])


In [83]:
# le 1 veut dire je veux que tu transforme mon tenseur en tenseur de dimension 1
# le -1 signifie, quelques soit le nombre de colonne et de raws, fait en sorte que ca fonctionne
y.view(1, -1)

tensor([[0.8352, 0.4473, 0.0355, 0.9228, 0.3016, 0.5993, 0.8486, 0.0842, 0.9758,
         0.8946]])

In [84]:
#autre exemple
x = torch.rand([17,5])
print(x)

tensor([[0.8589, 0.9774, 0.1267, 0.4872, 0.6009],
        [0.9181, 0.4921, 0.8708, 0.4466, 0.8675],
        [0.9320, 0.3127, 0.9883, 0.2780, 0.3676],
        [0.2537, 0.2133, 0.6165, 0.5572, 0.9587],
        [0.4082, 0.3598, 0.3268, 0.8912, 0.4331],
        [0.4744, 0.4663, 0.8735, 0.6122, 0.0397],
        [0.6109, 0.1888, 0.2805, 0.3954, 0.2455],
        [0.0237, 0.3705, 0.7930, 0.1520, 0.1921],
        [0.6145, 0.2284, 0.1437, 0.8138, 0.2041],
        [0.9346, 0.6814, 0.1385, 0.4030, 0.6243],
        [0.2064, 0.9423, 0.2886, 0.8816, 0.9937],
        [0.7003, 0.7847, 0.5340, 0.3842, 0.4546],
        [0.0722, 0.7205, 0.4217, 0.0499, 0.3765],
        [0.0804, 0.4982, 0.7653, 0.7028, 0.7902],
        [0.6298, 0.2399, 0.3113, 0.6215, 0.9448],
        [0.5032, 0.9564, 0.8843, 0.1366, 0.3695],
        [0.3311, 0.1262, 0.4707, 0.9563, 0.6541]])


In [85]:
x.view(1, -1)

tensor([[0.8589, 0.9774, 0.1267, 0.4872, 0.6009, 0.9181, 0.4921, 0.8708, 0.4466,
         0.8675, 0.9320, 0.3127, 0.9883, 0.2780, 0.3676, 0.2537, 0.2133, 0.6165,
         0.5572, 0.9587, 0.4082, 0.3598, 0.3268, 0.8912, 0.4331, 0.4744, 0.4663,
         0.8735, 0.6122, 0.0397, 0.6109, 0.1888, 0.2805, 0.3954, 0.2455, 0.0237,
         0.3705, 0.7930, 0.1520, 0.1921, 0.6145, 0.2284, 0.1437, 0.8138, 0.2041,
         0.9346, 0.6814, 0.1385, 0.4030, 0.6243, 0.2064, 0.9423, 0.2886, 0.8816,
         0.9937, 0.7003, 0.7847, 0.5340, 0.3842, 0.4546, 0.0722, 0.7205, 0.4217,
         0.0499, 0.3765, 0.0804, 0.4982, 0.7653, 0.7028, 0.7902, 0.6298, 0.2399,
         0.3113, 0.6215, 0.9448, 0.5032, 0.9564, 0.8843, 0.1366, 0.3695, 0.3311,
         0.1262, 0.4707, 0.9563, 0.6541]])

In [51]:
# connaitre la taille des inputs de sortie des maxpool par exemple avant la FC
# Contrairement aux couches conv dans pyTorch, où l'on a pas besoin de spécifier la taille d'entrée, 
# on doit le spécifier pour les couches entièrement connectées (FC layers).
# Ainsi, lorsque vous définissez la dimension d'entrée de la première couche linéaire, 
# vous devez savoir quelle est la taille des images que vous alimentez

class Print(nn.Module):
    def forward(self, x):
        #print((x.size())
        return x.view(x.size(0),-1)
a = Print()    
    

In [70]:
# On reprend un reseau de neurones simple mais en y ajoutant des convolutions
class Brain(nn.Module):
    def __init__(self):
        super().__init__()
        # ds cet ordre pr la Conv2D -> channel, filtres (nombre d outputs features) 
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        # 6 represente le nbre de feature en output
        self.conv2 = nn.Conv2d(32, 6, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        # 
        self.fc1 = nn.Linear(384, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 10)
        
    def forward(self, x):
        
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        
        # D'où vient ce 8*8*6
        # En keras ou tensorflow il exist la methode flatten
        # Mais comment faire en pytorch?
        # source: http://cs231n.github.io/convolutional-networks/#pool
        # https://pytorch.org/docs/stable/nn.html?highlight=maxpool2#torch.nn.Conv2d
        # https://pytorch.org/docs/stable/nn.html?highlight=maxpool2#torch.nn.MaxPool2d
        
        out = a.forward(out)
        print(out.shape)
        
        #out = out.view(-1, 8*8*6)
        #a.forward(out)
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out

model = Brain()
print(model)

Brain(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): Tanh()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): Tanh()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=384, out_features=32, bias=True)
  (act3): Tanh()
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)


In [71]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            loss_train += loss.item()
            
        
        print('{} Epoch {}, Training loss {}'.format(
            datetime.datetime.now(), epoch, float(loss_train)))

In [72]:
model = Brain()
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

In [73]:
training_loop(n_epochs = 100,optimizer = optimizer,model = model,loss_fn = loss_fn,train_loader = train_loader)

torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size

KeyboardInterrupt: 

In [ ]:
for loader in [train_loader, val_loader]:
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in loader:
            outputs = model(imgs)
            _, predicted = torch.max(outputs, dim=1)
            total += labels.shape[0]
            correct += int((predicted == labels).sum())
    
    print("Accuracy: %f" % (correct / total))

In [11]:
torch.save(model.state_dict(), data_path + 'CIFAR_10.pt')

In [99]:
# faire une prediction sur une image non vue et printer cette prediction
model_entraine = Brain()
model_entraine.load_state_dict(torch.load(data_path + 'CIFAR_10.pt'))


<All keys matched successfully>

### Le même modèle mais avec un resnet block

In [ ]:
# definition dun block Resnet



In [ ]:
# On reprend un reseau de neurones simple mais en y ajoutant des convolutions
class BrainALaResnet(nn.Module):
    def __init__(self):
        super().__init__()
        # ds cet ordre pr la Conv2D -> channel, filtres (nombre d outputs features) 
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 8, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 10)
        
    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * 8 * 8)
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out

model = Brain()
print(model)

In [ ]:
# rajouter: 
# regularization
# dropout
# batch normalization
# data augmentation
# transfer learning


In [ ]:
# resnet Block
# data_augmentation
# object_detection    